# **KW AI Hackathon - 컴퓨터 비전 AI 경진대회**



## - 팀 04 씽크빅데이터 - 

씽크빅데이터 팀은 2학년으로 구성된 팀으로써, 본격적인 대회 참가 및 프로젝트는 이번이 처음이었습니다.

대회 진행 중 정말 많은 시행착오와 아쉬운 점이 존재했지만, 이를 해결하는 과정 속에서 많이 성장할 수 있었다고 생각합니다.


향후 기회가 된다면 전공 지식을 더 쌓은 뒤 꼭 재도전해보고 싶습니다!

모두들 정말 고생 많으셨습니다!


Framework는 Keras, 환경은 Google Colab pro+(GPU) 이며, weight와 pretrained 데이터셋은 구글 드라이브 링크 공유하겠습니다.

weight, pretrained 링크 = https://drive.google.com/drive/folders/1vyRkWBXVoTRD3XCfEYb5oArBk0Xa57AF?usp=sharing

pretrained weight 링크 = https://www.kaggle.com/datasets/sreevishnudamodaran/efficientnetv2-tfhub-weight-files/discussion

--------------------------------------------------------------------------------

중간 평가 = 0.8737538462 -> 최종 평가 0.8781846154로 점수의 상승은 있었지만,

이는 단순 이전 중간 평가의 모델에 TTA(Test Time Augmentation)를 적용한 score입니다.

따라서 저번 중간 코드 중 추론 부분에서의 코드만 일부 수정되었습니다.

(제출 시간에 쫓겨 제대로 된 결과물을 뽑아내지 못했던 중간 평가의 교훈으로 코드를 대폭 수정 후 최종 평가때는 학습을 미리 진행하였으나, 제출 직전 vaildset generator에 큰 규모의 augmentation이 적용되었다는 것을 확인하였습니다. 

이에 따라 early stop과 learning rate scheduler가 정확한 시점에 작동하지 못하여 최종적으로 점수 개선이 되지 않았기에 우선은 제출한 점수에 맞는 본 코드를 공유하게 되었습니다.)

따라서 원래 수정사항은
- (model1 = efficientnet_v2_M_21k, model2 = efficientNet_v2_S_21k-ft1k) 

 -> (model1 = efficientnet_v2_M_21k, model2 = efficientNet_v2_M_21k-ft1k)
- train augmentation (90 degree rotation -> rotation_range=90)

- (Adam optimizer(lr=0.001) -> AdamW optimizer(lr=0.001,decay=0.0001)
- Kfold (split 3 -> split 5)

등이 있었지만 반영되지 못햇습니다.

아래는 원래 반영 예정본이 반영된 코드입니다.

추후 학습 자료로 남겨두기 위해 공유합니다.

https://colab.research.google.com/drive/1uLmFaFcYtrpxSw3eTS7yFf-ks2QYjggm?usp=sharing

# Try



실험은 대부분 LAB이라는 노트북을 따로 만든 뒤, 실험 결과에 따라 기록할 만한 가치가 없는 결과는 기록하지 않고, 다양한 비교가 필요하다고 생각되는 경우 결과물들을 엑셀에 저장하는 방식으로 진행되었습니다.

우선, 아래의 코드로 각 정답별 데이터가 얼추 비슷한 수로 존재하는 것을 확인하고, 최대한 균등하게 정답이 분포되어있는 sample을 추출하였습니다.(비율 조정)

In [ ]:
'''
sklearn.model_selection.train_test_split

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(train, train['label'], test_size=0.8, random_state=42) # train, vaild에 쓸 데이터와 test에서 쓸 데이터 분리
test_set_data, X_test, test_set_label, y_test = sklearn.model_selection.train_test_split(X_test, y_test, test_size=0.9, random_state=42) # test 사이즈 조절
train_set_data, valid_set_data, train_set_label, valid_set_label = sklearn.model_selection.train_test_split(X_train, y_train, test_size=0.25, random_state=42) # train과 vaild 데이터 분리

print(train.sum())
print(train_set_data.sum())
print(valid_set_data.sum())
print(test_set_data.sum())
'''

또한, 학습한 모델의 test_set에 대해 evaluate로 내부 스코어를 확인하고, 

이에 따른 predict 값을 저장하여 test_set에 대한 모델의 정답지(mark) DataFrame과 원래의 label 값(dap)DataFrame을 비교하여 각 class별 전체 일치율(def class_proba)과 실제 해당 알파벳이 존재할 때 모델이 이를 맞추는 비율(def acc_proba)을 확인하였습니다.

(본 계획은 빠른 모델링 후 이를 참고하여 threshhold 또는 정답률이 낮은 알파벳을 trainset에서 포토샵으로 재가공한 뒤 특정 문자들만 추가 학습시킬 계획이었으나 시간 관계상 진행은 못했습니다)

In [ ]:
'''
result = model.evaluate(test_gen_lab)
print(result)

expect = model.predict(test_gen_lab)

mark = test_set_data.drop(columns=['label','path']) # 모델이 예측한 정답 추출, pd.compare 위해 column조정 
for i in range(0,len(mark)) :
    mark.iloc[i,1:] = (expect[i] > 0.5).astype(int) # 확률값이 0.5보다 크다면 1, 작다면 0으로 할당
# mark.drop(columns='index')
dap = test_set_data.drop(columns=['label','path']) # 원래 정답 추출, pd.compare 위해 column조정 


def acc_proba(mark, dap, j) :
  check = dap.compare(mark, align_axis=1, keep_equal=True) # 모델이 예측한 정답과 실제 정답 compare 후 일치하는 것들 반환
  tmp_1 = (check[chr(j)].self + check[chr(j)].other)
  tmp_2 = tmp_1==2 #
  right_count = tmp_2.sum() # 실제로도 있고, 모델이 예측한것도 있다고 판단한 경우
  proba = right_count / dap[chr(j)].sum()

  return proba

  def class_proba(mark, dap, j) :
  check = dap.compare(mark, align_axis=0)
  right_count = check[chr(j)].isnull().sum()/2 # 전체 중 정답과 모델 예측값이 완전히 일치하는 경우 (원래 답에 없는 경우 포함) 
  full_count = len(test_set_data)
  proba = right_count / full_count

  return proba

for j in range(97,123) :
  print('{}의 일치률 : '.format(chr(j)) + str(acc_proba(mark,dap,j)) + '    {}의 클래스 정답률 : '.format(chr(j)) + str(class_proba(mark,dap,j)))
'''

실험을 통해 얻은 결론을 요약하자면 다음과 같습니다.

- EfficientNet_V2 모델들은 대부분 lr=0.001(default) 값에서 괜찮은 성능을 보인다.

- dropout(0.5) 층을 추가하는 것 역시 점수 향상에 도움이 된다.
- 2 종류의 pretrain weight 중 21k ft1보다는 21k 자체가 점수 향상에 도움이 되며, 모델 역시 무거운 모델이 대체적으로 결과가 괜찮다.
- GAUSSIAN BLUR와 BINARY, OTSU 등 noise와 관련된 전처리 및 grid distortion, zoom 등의 왜곡 및 부분 확대 역시 큰 점수 개선은 나타나지 않는다.
- rotation은 점수 향상에 도움이 되나, 90도씩 회전보다 ImageDataGenerator의 rotation_range=90 augmentation이 val_loss 감소 및 val_acc 향상 측면에서 큰 도움이 된다. (최종 평가때 반영 예정이었습니다.)
- Adam 자체 optimizer보다는 AdamW optimizer로 가중치를 일부 삭제하는 것 역시 오버피팅 대비 및 낮은 val_loss를 얻는 데 도움이 된다. (최종 평가때 반영 예정이었습니다.)
- 중간 평가의 결론을 종합하면 모델 조정만으로 나오는 점수는 87~88 정도가 최대인 것 같고, 이후는 데이터셋 가공 및 전처리의 문제가 될 것 같다. 

전반적인 코드 작성 및 모델, 튜닝은 위와 같은 결론을 바탕으로 진행되었습니다. 

# Version

In [ ]:
import sys
import tensorflow as tf
import keras
import sklearn
import numpy
import pandas
sys.version

!python --version
print('TensorFlow ' +  tf.__version__)
print('Keras ' + keras.__version__)
print('Sklearn ' + sklearn.__version__)
print('Numpy ' + numpy.__version__)
print('Pandas ' + pandas.__version__)

Python 3.7.14
TensorFlow 2.8.2
Keras 2.8.0
Sklearn 1.0.2
Numpy 1.21.6
Pandas 1.3.5


# Google drive에서 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import output
# !cp 파일1 파일2 # 파일1을 파일2로 복사 붙여넣기
!cp "/content/drive/MyDrive/kw_cv_comp/dataset/comp_data/235697_제 2회 컴퓨터 비전 학습 경진대회_data.zip" "data_2.zip" # 데이터셋 파일 경로 지정
# 위 파일 경로를 google drive 중 대회 데이터셋에 해당하는 경로로 지정하시면 colab 환경에서 그대로 작동합니다.

# data_2.zip을 현재 디렉터리에 압축해제
!unzip "data_2.zip"

In [ ]:
from google.colab import output
# 현재 디렉터리에 dirty_mnist라는 폴더 생성
!mkdir "./dirty_mnist"
#dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist/"
# 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist"
#test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"
# 출력 결과 지우기
output.clear()

# Library Import

In [ ]:
from google.colab import output
from keras import backend as K 

import os
import zipfile

import pandas as pd
import numpy as np

import cv2
from tqdm import tqdm

import random as python_random # 시드값 고정
seed_num = 42
np.random.seed(seed_num)
python_random.seed(seed_num)
tf.random.set_seed(seed_num)


#  Dataset

In [ ]:
train = pd.read_csv('/content/dirty_mnist_2nd_answer.csv')
test = pd.read_csv('/content/sample_submission.csv')
sub = pd.read_csv('/content/sample_submission.csv')

In [ ]:
col_list = np.array(train.iloc[:,1:].columns) # a,b,c,d,e.. 등의 column들의 이름을 1차원 넘파이로 표현
num_list = np.array(train.iloc[:,1:]) # 각 데이터가 갖는 label에 대한 값들을 binary 형식 1차원 넘파이로 표현
multi_list = num_list * col_list # 모델의 target data로 사용하기 위해 각 이미지에 대한 label 값을 1차원 넘파이로 저장
label_pd = pd.DataFrame(multi_list).apply(lambda x : ' '.join(x),axis=1) 
label_pd = pd.Series(label_pd) 
train['label'] = label_pd # train data에 각 index에 해당하는 1차원 넘파이로 표시된 label 값을 column으로 추가
train['label'] = train['label'].apply(lambda x : x.split()) # label 값 중 1이 아닌 값들이 ''로 나타나므로 따로 처리
train

In [ ]:
train['path'] = '/content/dirty_mnist/' + train['index'].apply(lambda x : str(x).zfill(5)) + '.png'
train

In [ ]:
test['path'] =  '/content/test_dirty_mnist/' + test['index'].apply(lambda x : str(x).zfill(5)) + '.png'
test

In [ ]:
display(train,test) # Path 및 Target 값 구현 확인

Multi-Label 역시 ImageDataGenerator에서 처리 가능한 것을 확인했기에, flow_from_dataframe을 이용하여 전처리하고자 train에 'label' column을 추가했습니다.

# Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet_v2 import *
from tensorflow.keras.callbacks import *
import albumentations as al

In [ ]:
def rotate_image(image):
   return np.rot90(image, np.random.choice([-1, 0, 1, 2])) # 90도씩 회전하여 fill_mode가 적용되지 않는 rotation

In [ ]:
idg_rotate = ImageDataGenerator(preprocessing_function = rotate_image)
idg_nomal = ImageDataGenerator()

글자 이미지 특성상 심한 왜곡이나 변형이 주어질 시, general한 결과를 얻기 어려울 것이라고 판단했습니다.

따라서, flip과 range rotate, zoom 등 fill_mode가 적용되거나 주어진 label에 해당하는 정보가 왜곡될 수 있는 augmentation은 최대한 지양하고, 단순한 rotation만 추가하였습니다. 

추가로, noise가 적용된 data인 점을 고려하여 GAUSSIAN BLUR와 BINARY, OTSU 등의 필터 역시 적용해 보았으나 유의미한 점수 개선은 나타나지 않았습니다.

# Train

모델은 모두 EfficientNet_V2 모델을 사용하였습니다. 

pretrained 모델이 허용된 대회인 만큼 이미지 분류에서 강세를 보이는 EfficientNet 모델을 사용하려 했으나, Google Colab pro+ 환경에서도 EfficientNet 상위 모델은 메모리 및 학습 시간의 문제로 사용이 어려워 ResNet, Inception 등 다양한 모델을 고려하던 중 평상시 궁금했던 EfficientNet_V2가 눈에 들어오게 되었고, 최종적으로 해당 모델을 사용하게 되었습니다.

관련 논문인 'EfficientNetV2: Smaller Models and Faster Training'의 안내대로 학습 시간에 비해 좋은 결과를 내주었지만, progressive learning 방식을 채택한 만큼 lr 설정이 까다로워 실험을 통한 manual search 방식으로 하이퍼 파라미터를 조정했습니다.

추후 여유가 된다면 다양한 optimizer와의 조합을 찾아볼 가치가 있을 것 같습니다.

학습은 모두 3 Fold Validation을 기반으로 각 Fold 별 Best 모델을 저장한 뒤, Inference에서 TTA를 적용하였습니다.

※ 본 코드는 하나의 노트북으로 작성되었지만, 실제 학습은 다수의 노트북을 이용하여 Fold별로 학습시킨 뒤 모델을 따로 저장했습니다. 

# MODEL 1 - EfficientNet_V2 S

In [ ]:
kf = KFold(n_splits=3, shuffle=True,random_state=seed_num) # 3 Fold Validation

for fold_var, (train_index, valid_index) in enumerate(kf.split(train, train['label']), 1) :

  if fold_var < 3 :
      
    x_train = train.iloc[train_index]
    x_valid = train.iloc[valid_index]
    train_gen = idg_rotate.flow_from_dataframe(x_train, x_col='path', y_col='label', target_size=(256,256), batch_size=32)
    valid_gen = idg_rotate.flow_from_dataframe(x_valid, x_col='path', y_col='label', target_size=(256,256), batch_size=32) 
    # 대부분은 vaildset에 augment를 적용하지 않지만, 대부분의 이미지들이 이미 rotate, filp등이 적용된 상태였고, ReduceLROnPlateau로 적절한 시점에서의 lr 조정을 위해 실험해본 결과 지금과 같이 단순한 90도 rotation 적용은 최종 정확도에 약간의 긍정적인 영향을 주었기에 이를 적용하였습니다.
    # 최종 평가 학습 부분에서는 90도 회전이 아닌 -90도~90도까지의 다양한 각도 회전 augmentation을 적용하였기에 분명히 augmentation을 적용하지 않았어야 했는데 이 부분을 수정하지 않아 문제가 되었습니다
    model = Sequential()
    model.add(EfficientNetV2S(include_top=False, weights='/content/drive/MyDrive/kw_cv_comp/pretrain/efficientnetv2-s-21k_notop.h5', pooling='avg'))
    # weights는 https://www.kaggle.com/datasets/sreevishnudamodaran/efficientnetv2-tfhub-weight-files/discussion 의 'EfficientNetV2 TFHub Weight Files'의 s-21k 공식 ImageNet weight활용 (경로지정)
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(Dense(26, activation='sigmoid'))
    model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = 'binary_accuracy') # Adam lr = 0.001(default)
    es = EarlyStopping(patience=3, restore_best_weights=True) 
    rl = ReduceLROnPlateau(patience=2, verbose=1, min_lr=0.000001) 
    checkpoint_filepath_md1 = '/content/drive/MyDrive/kw_cv_comp/model/model1/' # 폴드별 모델1 저장 경로 설정('/content/'도 괜찮습니다.)
    cb = ModelCheckpoint(filepath=checkpoint_filepath_md1 + 'fold{}_efficientNet_S_best.h5'.format(str(fold_var)), save_best_only=True, exist_ok=True)
    model.fit(train_gen, validation_data=valid_gen, epochs=100, callbacks=[es,cb,rl]) 

  else :
    break

K.clear_session()

3 Fold Validation을 진행하려 했으나, 최종 결과를 위해 학습하던 도중 코랩의 런타임 문제로 인해 마지막 3 Fold의 Best 모델을 추출하지 못하고 Fold 1과 2만 저장하여 최종 결과를 도출했습니다.

따라서 본 코드 역시 이를 반영하여 2 Fold까지만 모델을 저장했습니다.

#  MODEL 2 - EfficientNet_V2 M

In [ ]:
kf_2 = KFold(n_splits=3, shuffle=True,random_state=seed_num) # 3 Fold Validation

for fold_var2, (train_index, valid_index) in enumerate(kf.split(train, train['label']), 1) :

  # if fold_var < 2 : 실제로는 조건문 안에서 fold 별로 학습

    Adam = tf.keras.optimizers.Adam(learning_rate=0.0005) # Adam lr = 0.0005
    x_train = train.iloc[train_index]
    x_valid = train.iloc[valid_index]
    train_gen = idg_rotate.flow_from_dataframe(x_train, x_col='path', y_col='label', target_size=(256,256), batch_size=32)
    valid_gen = idg_rotate.flow_from_dataframe(x_valid, x_col='path', y_col='label', target_size=(256,256), batch_size=32)
    model2 = Sequential()
    model2.add(EfficientNetV2M(include_top=False, weights='/content/drive/MyDrive/kw_cv_comp/pretrain/efficientnetv2-m-21k-ft1k_notop.h5', pooling='avg'))
    # weights는 https://www.kaggle.com/datasets/sreevishnudamodaran/efficientnetv2-tfhub-weight-files/discussion 의 'EfficientNetV2 TFHub Weight Files'의 m-21k-ft1 공식 ImageNet weight활용 (경로지정)
    model2.add(tf.keras.layers.Dropout(0.5))
    model2.add(Dense(26, activation='sigmoid'))
    model2.compile(optimizer=Adam, loss='binary_crossentropy', metrics='binary_accuracy')
    es = EarlyStopping(patience=3, restore_best_weights=True) 
    rl = ReduceLROnPlateau(patience=2,verbose=1, min_lr=0.000001) 
    checkpoint_filepath_md2 = '/content/drive/MyDrive/kw_cv_comp/model/model2/' # 폴드별 모델2 저장 경로 설정('/content/'도 괜찮습니다.)
    cb = ModelCheckpoint(filepath=checkpoint_filepath_md2 + 'fold{}_efficientNet_M_best.h5'.format(str(fold_var2)), save_best_only=True, exist_ok=True)
    model2.fit(train_gen, validation_data=valid_gen, epochs=100, callbacks=[es,cb,rl])

  # else :
  #   break

K.clear_session()

# Inference & model import

이전에 저장했던 각 model의 Fold 별 Best 모델을 불러와서 TTA를 적용하였습니다.
TTA는 기존의 augmentation과 동일하게 90도씩 회전하는 augmentation만 적용하였습니다.

In [ ]:
from tensorflow import keras
from tensorflow.python.keras.models import load_model


test_gen = idg_nomal.flow_from_dataframe(test, x_col='path', y_col=None, target_size=(256,256), shuffle=False, class_mode=None)


model1_1fold = keras.models.load_model(checkpoint_filepath_md1 + 'fold1_efficientNet_S_best.h5') # model 1의 1fold Best 모델
model1_2fold = keras.models.load_model(checkpoint_filepath_md1 + 'fold2_efficientNet_S_best.h5') # model 1의 2fold Best 모델


model2_1fold = keras.models.load_model(checkpoint_filepath_md2 + 'fold1_efficientNet_M_best.h5') # model 2의 1fold Best 모델
model2_2fold = keras.models.load_model(checkpoint_filepath_md2 + 'fold2_efficientNet_M_best.h5') # model 2의 2fold Best 모델
model2_3fold = keras.models.load_model(checkpoint_filepath_md2 + 'fold3_efficientNet_M_best.h5') # model 2의 3fold Best 모델


# TTA

In [ ]:
def generator_predict(model, test_generator, test_aug_generator, batch_size=32): # TTA를 위한 함수 정의

    pred_list = []
    pred_list.append(model.predict_generator(test_generator, # 기본적인 결과값
                                             steps=train_step, 
                                             verbose=1
                                            ))

    for i in range(9):
        pred_list.append(model.predict_generator(test_aug_generator, # rotate augmentation 적용된 결과값
                                                 steps=train_step, 
                                                ))

    predicted = np.mean(pred_list, axis=0)
    return predicted 

In [ ]:
train_step = np.ceil(len(test) / 32)
model_list = [model1_1fold, model1_2fold, model2_1fold, model2_2fold, model2_3fold]
model_pred = []
for i, name in enumerate(model_list):
    test_generator = idg_nomal.flow_from_dataframe(test, x_col='path', y_col=None, target_size=(256,256), shuffle=False, class_mode=None) # 기본 ImageDataGenerator
    test_aug_generator = idg_rotate.flow_from_dataframe(test, x_col='path', y_col=None, target_size=(256,256), shuffle=False, class_mode=None) # rotate augmentation 적용된 ImageDataGenerator
    
    print('{} model predict'.format(name))
    model_pred.append(generator_predict(model=name, test_generator=test_generator, test_aug_generator=test_aug_generator))

In [ ]:
predict = np.mean(model_pred, axis=0) # 5모델의 평균 
predict

In [ ]:
tta_ensemble = pd.read_csv('/content/sample_submission.csv') # sample_submission 파일 경로

for i in range(0,5000) :
    tta_ensemble.iloc[i,1:] = (predict[i] > 0.5).astype(int) # 확률값이 0.5보다 크다면 1, 작다면 0으로 할당
tta_ensemble

tta_ensemble.to_csv('Thinkbigdata_tta_ensemble.csv',index=0)